# JellyDB Live Demonstration
### Presented by Team JeLLY-B
### ECS 165A 

In [126]:
#!python -m JellyDB.test
import JellyDB

# Basic Functionality 

At its core, JellyDB effectively stores data in a simple, L-Store style, columnar storage database. 

**Creating Databases**

By calling 'create_table()' we're able to create a new database based on a user-called key and length.

Suppose we survey individuals for their three favorite numbers: 

In [127]:
db = Database()

fav_numbers = db.create_table('fav_numbers', 4, 0)

Or if we want to record winning lottery numbers by day:

In [128]:
winning_lotto_numbers = db.create_table('winning_lotto', 7, 0)

We're able to create a space to store that data. 

**Inserting and Referencing Table Information**

Once we've built a table, we can insert new records and call them. For our survey responses, we'll input the response number and each respective set of three responses: 

In [129]:
query = Query(fav_numbers)

response1 = [1, 5, 6, 7]
response2 = [2, 25, 26, 27]
response3 = [3, 99, 98, 97]

query.insert(*response1)
query.insert(*response2)
query.insert(*response3)

Should we (accidentally) try to insert the respondant's information again we'll throw an error: 

In [130]:
query.insert(*response3) # value already exists

Exception: Value already exists

This helps to maintain the integrity of the database. 

We can see that the records were successfully inserted on the first round.


In [ ]:
for responses in [1, 2, 3]:
            # Get all columns
            print(query.select(responses, [1, 1, 1, 1]))


**Updating Records**

To update a set of responses, however, we can simply call the update function for a specific key. 

If we want to update the third set of responses: 

In [ ]:
query.update(3, *(None, 12, 20, 24))

print("Updated Values: ")
query.select(3,[1,1,1,1])

**Deleting Records**

In accordance with good database structure, the user is also able to delete existing records. 

In [132]:
query.delete(3)

Exception: No index exists for column 0

## Error Handling

While we've shown that we can handle duplication errors on insertion, we've also accounted for several other user errors: 

 - Attempting to delete a non-existent record
 - Attempting to update a deleted record 